In [36]:
import os
import re
import pandas as pd
import numpy as np
import glob
import nltk
import unicodedata
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
nltk.download('wordnet')
from sklearn.svm import SVC

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
# load data
file_path = r"../datasets/"
train_file = r"train.csv"
test_file = r"test.csv"
subm_file = r"sample_submission.csv"

train = pd.read_csv(file_path+train_file)
test = pd.read_csv(file_path+test_file)

print(train.shape)
train.head()

(7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [22]:
train["length"] = train["text"].apply(lambda x : len(x))
test["length"] = test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(train["length"].describe())
print()

print("Test Length Stat")
print(test["length"].describe())

Train Length Stat
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64

Test Length Stat
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64


In [23]:
train.location.value_counts()

location
USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
Some pum pum             1
Paulton, England         1
SEA Server               1
todaysbigstock.com       1
???????? ?????????.      1
Name: count, Length: 3341, dtype: int64

In [24]:
train.keyword.value_counts()

keyword
fatalities               45
armageddon               42
deluge                   42
harm                     41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 221, dtype: int64

In [25]:
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [26]:
def clean_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Remove mentions (@username)
    tweet = re.sub(r'@\w+', '', tweet)
    
    # Remove hashtags (#)
    tweet = re.sub(r'#', '', tweet)
    
    # Remove hashtags preceded by spaces
    tweet = re.sub(r' #', ' ', tweet)
    
    # Remove dashes (-)
    tweet = re.sub(r'-', '', tweet)
    
    # Remove special characters and punctuations (except alphanumeric and spaces)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    
    # Remove non-ASCII characters and emojis
    tweet = ''.join(char for char in tweet if char in string.printable)
    
    # Remove punctuation using Unicode categories
    tweet = ''.join(char for char in tweet if not unicodedata.category(char).startswith('P'))
    
    # Convert to lowercase
    tweet = tweet.lower()
    
    # Remove extra whitespaces
    tweet = ' '.join(tweet.split())
    
    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)
    
    # Remove single characters (e.g., 'a', 'b', 'c')
    tweet = re.sub(r'\b\w\b', '', tweet)
    
    # Remove multiple spaces
    tweet = re.sub(r'\s+', ' ', tweet)
    
    # Remove the word "nan"
    tweet = re.sub(r'\bnan\b', '', tweet)

    tweet = emoji_pattern.sub(r'', tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tweet = ' '.join(word for word in tweet.split() if word not in stop_words)
    
    return tweet

In [27]:
train['text']=train.text.apply(clean_tweet)
train.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1,69
1,4,NaN,NaN,forest fire near la ronge sask canada,1,38
2,5,NaN,NaN,residents asked shelter place notified officer...,1,133
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,65
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1,88


In [48]:
lemma=WordNetLemmatizer()

def preprocessdata(text):
    text= ' '.join(lemma.lemmatize(word) for word in text.split(' '))
    return text

train['text']=train['text'].apply(preprocessdata)
train.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,deed reason earthquake may allah forgive u,1,69
1,4,NaN,NaN,forest fire near la ronge sask canada,1,38
2,5,NaN,NaN,resident asked shelter place notified officer ...,1,133
3,6,NaN,NaN,people receive wildfire evacuation order calif...,1,65
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfire pour...,1,88


In [49]:
#split data
X=train["text"]
y=train['target'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
#using TFIDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

In [51]:
#using Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

y_pred_nb = nb_classifier.predict(X_val_tfidf)
f1 = f1_score(y_val, y_pred_nb, average='macro')
print(f'F1 Score: {f1 * 100:.2f}%')
report_nb = classification_report(y_val, y_pred_nb)
print("SVM Classification Report:\n", report_nb)

F1 Score: 79.17%
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.90      0.84       874
           1       0.83      0.68      0.74       649

    accuracy                           0.80      1523
   macro avg       0.81      0.79      0.79      1523
weighted avg       0.81      0.80      0.80      1523



In [52]:
#using SVM
svm_model=SVC(kernel="linear", probability=True)
svm_model.fit(X_train_tfidf, y_train)

y_pred_svm = svm_model.predict(X_val_tfidf)
f1 = f1_score(y_val, y_pred_svm, average='macro')
print(f'F1 Score: {f1 * 100:.2f}%')
report_svm = classification_report(y_val, y_pred_svm)
print("SVM Classification Report:\n", report_svm)

F1 Score: 78.31%
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.85      0.82       874
           1       0.78      0.71      0.74       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523

